In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(response.text, features="html.parser")

table_data = soup.select('table.wikitable.sortable > tbody > tr')
theads = table_data[0].select('th')
trows = table_data[1:]

columns = []
for thead in theads:
    columns.append(thead.get_text().strip())

data_rows = [[] for col in columns]
for row in trows:
    tds = row.select('td')
    for idx, td in enumerate(tds):
        data_rows[idx].append(td.get_text().strip())

df = pd.DataFrame()
for idx, col in enumerate(columns):
    df[col] = data_rows[idx]

df.to_csv(r'toronto_neighbor.csv', index=False)

In [2]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
# !conda install -c conda-forge geocoder --yes

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

address = 'Parkwoods, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.7611243 -79.3240594


In [5]:
def get_pos(address):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    if location is None:
        return [0,0]
    latitude = location.latitude
    longitude = location.longitude
    print(address, latitude, longitude)
    return [latitude, longitude]

In [6]:
df['Neighbourhood_Toronto'] = df['Neighbourhood'] + ', Toronto'
new_cols = df['Neighbourhood_Toronto'].apply(lambda nbh: get_pos(nbh))

Parkwoods, Toronto 43.7611243 -79.3240594
Victoria Village, Toronto 43.732658 -79.3111892
Harbourfront, Toronto 43.6400801 -79.3801495
Regent Park, Toronto 43.6607056 -79.3604569
Lawrence Heights, Toronto 43.7227784 -79.4509332
Lawrence Manor, Toronto 43.7220788 -79.4375067
Islington Avenue, Toronto 43.7192407 -79.5570574
Rouge, Toronto 43.8049304 -79.1658374
Malvern, Toronto 43.8091955 -79.2217008
Don Mills North, Toronto 43.737178 -79.3434514
Woodbine Gardens, Toronto 43.7120785 -79.3025673
Parkview Hill, Toronto 43.7062977 -79.3219073
Ryerson, Toronto 43.65815275 -79.37908010263
Garden District, Toronto 43.6565019 -79.3771275
Glencairn, Toronto 43.7087117 -79.4406853
Cloverdale, Toronto 43.633637 -79.5497447
Islington, Toronto 43.6453376 -79.52487
Martin Grove, Toronto 43.653963 -79.387207
Princess Gardens, Toronto 43.6404664 -79.3912237
West Deane Park, Toronto 43.6631995 -79.5685684
Highland Creek, Toronto 43.7901172 -79.1733344
Rouge Hill, Toronto 43.7802711 -79.1304992
Port Unio

In [58]:
df['Latitude'] = new_cols.apply(lambda x: x[0])
df['Longitude'] = new_cols.apply(lambda x: x[1])
df = df[df['Latitude'] != 0]
df = df[df['Longitude'] != 0]
df.reset_index(drop=True, inplace=True)
df.shape

(200, 6)

In [62]:
df.drop(['Neighbourhood_Toronto'], axis=1, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,0.000000,0.000000
1,M2A,Not assigned,Not assigned,0.000000,0.000000
2,M3A,North York,Parkwoods,43.761124,-79.324059
3,M4A,North York,Victoria Village,43.732658,-79.311189
4,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
